<img align="left" src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=100 style="padding: 20px"> 

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=160 style="padding: 20px">  

# CECI with RAIL stage - NOT FINISHED

**Contact author**: Heloisa da Silva Mengisztki ([heloisasmengisztki@gmail.com](mailto:heloisasmengisztki@gmail.com)) 

**Last verified run**: 2022-12-01 (YYYY-MM-DD) <br><br><br>

the main purpose of this notebook is to understand how we can pass data to a rail stage using ceci

In [ ]:
import os
import numpy as np
import pandas as pd
import qp
import tables_io
import ceci

import rail
from rail.core.utils import RAILDIR
from rail.core.data import TableHandle, PqHandle, ModelHandle, QPHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper

In [ ]:
CURR_DIR = os.getcwd()
CURR_DIR, RAILDIR

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

### Reading some sample

In [ ]:
headers=[
    "u_auto", "u_auto_err", "u_det", "u_det_err",
    "g_auto", "g_auto_err", "g_det", "g_det_err", 
    "r_auto", "r_auto_err", "r_det", "r_det_err", 
    "i_auto", "i_auto_err", "i_det", "i_det_err", 
    "z_auto", "z_auto_err", "z_det", "z_det_err", 
    "Y_auto", "Y_auto_err", "Y_det", "Y_det_err", 
    "z", "ra", "dec", "z_flags", "field_id", "number"
]

test_sample = pd.read_csv("/home/heloisamengisztki/DATA/des_sv_main_standard.valid", delimiter=" ", names=headers)
test_sample_path = "/home/heloisamengisztki/DATA/des_sv_main_standard.valid";

In [ ]:
test_sample.head()

In [ ]:
columns_map = {
"field_id": "id",
"ra": "coord_ra",
"dec": "coord_dec",
"g_auto": "mag_g_lsst",
"g_auto_err": "mag_err_g_lsst",
"r_auto": "mag_r_lsst",
"r_auto_err": "mag_err_r_lsst",
"i_auto": "mag_i_lsst",
"i_auto_err": "mag_err_i_lsst",
"u_auto": "mag_u_lsst",
"u_auto_err": "mag_err_u_lsst",
"Y_auto": "mag_y_lsst",
"Y_auto_err": "mag_err_y_lsst",
"z_auto": "mag_z_lsst",
"z_auto_err": "mag_err_z_lsst",
"z": "redshift"
}

## Executing with ceci

In [ ]:
# test 1 -> fails cannot find path to output file with ceci
def test1():
    col_remapper_test = ColumnMapper.make_stage(name='col_remapper_test', columns=columns_map)
    col_remapper_test(test_sample)
    return col_remapper_test
    

def test2():
    col_remapper_test = ColumnMapper.make_stage(name='col_remapper_test', columns=columns_map, input=test_sample) # ignores the key input
    col_remapper_test.run()
    return col_remapper_test

    
def test3():
    col_remapper_test = ColumnMapper.make_stage(name='col_remapper_test', columns=columns_map, input="keeey") 
    DS.add_data(key="input", data=test_sample, handle_class=PqHandle)
    col_remapper_test.run()
    return col_remapper_test

def test4():
    col_remapper_test = ColumnMapper.make_stage(name='col_remapper_test', columns=columns_map, input=test_sample_path) 
    DS.add_data(key="input", data=test_sample_path, handle_class=PqHandle)
    col_remapper_test.run()
    return col_remapper_test


In [ ]:
#col_remapper_test = test1() # test 1 -> fails cannot find path to "inprogress_output_col_remapper_test.pq" file with ceci
col_remapper_test = test2() # test 1 -> fails data size in the yaml
#col_remapper_test = test3() # test 1 -> fails cannot find path to "inprogress_output_col_remapper_test.pq" file with ceci
#col_remapper_test = test4()

col_remapper_test.config.to_dict()

In [ ]:
col_remapper_test.get_data("output")

In [ ]:
pipe = ceci.Pipeline.interactive() ## submódulo interativo de pipelines pelo ceci. Class MiniPipeline
pipe.add_stage(col_remapper_test)

In [ ]:
#help(ceci.Pipeline.interactive())
#help(pipe.add_stage)

In [ ]:
# esse input aqui é o que vai popular o input da nossa DS? 
pipe.initialize(dict(input=test_sample_path), dict(output_dir='.', log_dir='.', resume=False), None)
pipe.save('pipe.yml')